In [4]:
import os

folder_path = "../Datasets/SROIE_YOLO_TRAIN/val/images"

count = sum(1 for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)))
print("Số file trong thư mục:", count)


Số file trong thư mục: 126


In [3]:
from ultralytics import YOLO
import cv2
import os
import random

# 1. Đường dẫn model (File xịn nhất v2)
MODEL_PATH = r'..\train\runs\detect\sroie_yolov9c_finetune\weights\best.pt'

# 2. Chọn một ảnh bất kỳ từ tập Val để test
# (Hoặc bạn có thể thay bằng đường dẫn 1 file ảnh hóa đơn bất kỳ trên mạng tải về)
IMAGE_PATH = r'D:\ADMIN\Documents\Classwork\advance_cv_project\output_results\image_140_ocr.jpg'

print(f"Đang test trên ảnh: {IMAGE_PATH}")

# 3. Load Model
model = YOLO(MODEL_PATH)

# 4. Dự đoán
# save=True: Tự động lưu ảnh có vẽ box vào folder runs/detect/predict
# conf=0.4: Ngưỡng tự tin 40%
results = model.predict(source=IMAGE_PATH, save=True, conf=0.4) 

# 5. In thông tin ra màn hình
for result in results:
    boxes = result.boxes
    print(f"-> Tìm thấy {len(boxes)} vùng text.")
    
    # In tọa độ 5 box đầu tiên xem thử
    for i, box in enumerate(boxes[:5]):
        # Tọa độ [x1, y1, x2, y2]
        coords = box.xyxy[0].cpu().numpy().astype(int)
        conf = float(box.conf)
        print(f"   Box {i+1}: {coords} - Độ tin cậy: {conf:.2f}")

print("\n--- XONG ---")
print(f"Bạn hãy mở folder '{results[0].save_dir}' để xem ảnh kết quả nhé!")

Đang test trên ảnh: D:\ADMIN\Documents\Classwork\advance_cv_project\output_results\image_140_ocr.jpg

image 1/1 D:\ADMIN\Documents\Classwork\advance_cv_project\output_results\image_140_ocr.jpg: 1024x640 56 Texts, 883.1ms
Speed: 7.6ms preprocess, 883.1ms inference, 5.0ms postprocess per image at shape (1, 3, 1024, 640)
Results saved to D:\ADMIN\Documents\Classwork\advance_cv_project\notebooks\runs\detect\predict3
-> Tìm thấy 56 vùng text.
   Box 1: [185 555 265 581] - Độ tin cậy: 0.89
   Box 2: [180 595 322 634] - Độ tin cậy: 0.88
   Box 3: [178 628 324 665] - Độ tin cậy: 0.88
   Box 4: [175 672 320 704] - Độ tin cậy: 0.88
   Box 5: [192 439 271 463] - Độ tin cậy: 0.88

--- XONG ---
Bạn hãy mở folder 'D:\ADMIN\Documents\Classwork\advance_cv_project\notebooks\runs\detect\predict3' để xem ảnh kết quả nhé!


In [1]:
from ultralytics import YOLO
import cv2
import os
import json
import uuid
import urllib.parse

# --- CẤU HÌNH ---

# 1. Đường dẫn model best.pt (Hãy chắc chắn đường dẫn này đúng trên máy bạn)
MODEL_PATH = r'..\train\runs\detect\sroie_yolov9c_finetune\weights\best.pt'

# 2. Thư mục chứa ảnh đầu vào (Đã sửa theo yêu cầu của bạn)
INPUT_FOLDER = r'D:\ADMIN\Documents\Classwork\advance_cv_project\output_results'

# 3. Tên file JSON kết quả
OUTPUT_JSON = 'output_results_labelstudio.json'

# 4. Prefix đường dẫn Local Storage
# Dựa trên đường dẫn INPUT_FOLDER của bạn, tôi đã mã hóa nó theo chuẩn Label Studio
# (ADMIN\Documents... -> ADMIN%5CDocuments...)
# Lưu ý: Nếu Label Studio không load được ảnh, hãy kiểm tra lại dòng này trong Data Manager
LOCAL_STORAGE_PREFIX = "/data/local-files/?d=ADMIN%5CDocuments%5CClasswork%5Cadvance_cv_project%5Coutput_results%5C"

# --- XỬ LÝ ---

def main():
    # Kiểm tra model tồn tại
    if not os.path.exists(MODEL_PATH):
        print(f"LỖI: Không tìm thấy model tại {MODEL_PATH}")
        # Nếu đang chạy khác thư mục, hãy sửa lại đường dẫn model tuyệt đối, ví dụ:
        # MODEL_PATH = r'D:\Datasets\SROIE_YOLO_TRAIN\runs\detect\sroie_best_v2\weights\best.pt'
        return

    print(f"Loading model: {MODEL_PATH}")
    model = YOLO(MODEL_PATH)

    # Lấy danh sách ảnh
    if not os.path.exists(INPUT_FOLDER):
        print(f"LỖI: Không tìm thấy thư mục ảnh {INPUT_FOLDER}")
        return

    image_files = [f for f in os.listdir(INPUT_FOLDER) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    print(f"Tìm thấy {len(image_files)} ảnh trong 'output_results'. Đang xử lý...")

    ls_tasks = []

    for filename in image_files:
        image_path = os.path.join(INPUT_FOLDER, filename)

        # 1. Đọc ảnh để lấy kích thước gốc
        img = cv2.imread(image_path)
        if img is None: continue
        height, width = img.shape[:2]

        # 2. Dự đoán
        results = model.predict(source=image_path, conf=0.4, verbose=False)

        # 3. Tạo list kết quả
        result_items = []

        for box in results[0].boxes:
            # Tọa độ pixel
            coords = box.xyxy[0].cpu().numpy()
            x1, y1, x2, y2 = coords

            box_w = x2 - x1
            box_h = y2 - y1

            # Convert sang %
            x_pct = (x1 / width) * 100
            y_pct = (y1 / height) * 100
            w_pct = (box_w / width) * 100
            h_pct = (box_h / height) * 100

            region_id = str(uuid.uuid4())[:8]

            item = {
                "id": region_id,
                "from_name": "label",
                "to_name": "image",
                "type": "rectanglelabels",
                "original_width": width,
                "original_height": height,
                "image_rotation": 0,
                "value": {
                    "x": x_pct,
                    "y": y_pct,
                    "width": w_pct,
                    "height": h_pct,
                    "rotation": 0,
                    "rectanglelabels": ["Text"]
                }
            }
            result_items.append(item)

        # 4. Tạo URL ảnh chuẩn Label Studio
        # Nối Prefix + Tên file (được mã hóa nếu có dấu cách)
        encoded_filename = urllib.parse.quote(filename)
        image_url = LOCAL_STORAGE_PREFIX + encoded_filename

        # 5. Đóng gói JSON
        # Dùng "predictions" để bạn có thể xem và chỉnh sửa trên Label Studio
        task = {
            "data": {
                "image": image_url
            },
            "predictions": [{
                "model_version": "yolov8_v2_best",
                "score": 0.95,
                "result": result_items
            }]
        }
        ls_tasks.append(task)

    # 6. Xuất file
    with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
        json.dump(ls_tasks, f, ensure_ascii=False, indent=2)

    print(f"\n--- XONG! ---")
    print(f"File JSON đã lưu tại: {os.path.abspath(OUTPUT_JSON)}")
    print(f"Số lượng ảnh đã xử lý: {len(ls_tasks)}")

if __name__ == '__main__':
    main()

Loading model: ../train/runs/detect/sroie_best_v2/weights/best.pt
Tìm thấy 8 ảnh trong 'output_results'. Đang xử lý...

--- XONG! ---
File JSON đã lưu tại: D:\ADMIN\Documents\Classwork\advance_cv_project\notebooks\output_results_labelstudio.json
Số lượng ảnh đã xử lý: 8
